In [1]:
import torch

In [2]:
import torch.nn as nn

In [3]:
import numpy as np

In [4]:
from sklearn import datasets

In [5]:
from sklearn.preprocessing import StandardScaler

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
# input data preparation

In [8]:
bc = datasets.load_breast_cancer()

In [9]:
X, y = bc.data, bc.target

In [10]:
n_samples, n_features = X.shape

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

In [12]:
# scale

In [13]:
sc = StandardScaler()

In [14]:
X_train = sc.fit_transform(X_train)

In [15]:
X_test = sc.transform(X_test)

In [16]:
X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

In [17]:
y_train = y_train.view(y_train.shape[0],1)
y_test = y_test.view(y_test.shape[0],1)

In [18]:
# Model

In [19]:
class LogisticRegression(nn.Module):
    
    def __init__(self, n_input_features):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)
        
    def forward(self, x):
        y_predicted = torch.sigmoid(self.linear(x))
        return y_predicted
    
model = LogisticRegression(n_features)

In [20]:
# Loss
learning_rate = 0.01
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [21]:
# training loop

num_epochs = 100

for epoch in range(num_epochs):
    y_predicted = model(X_train)
    loss = criterion(y_predicted, y_train)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    if epoch % 10 == 0:
        print(f"epoch: {epoch+1}, loss = {loss.item():.4f}")
        
# Evaluation
with torch.no_grad():
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f"accuracy= {acc:.4f}")
        

epoch: 1, loss = 0.7785
epoch: 11, loss = 0.5973
epoch: 21, loss = 0.4929
epoch: 31, loss = 0.4260
epoch: 41, loss = 0.3793
epoch: 51, loss = 0.3447
epoch: 61, loss = 0.3179
epoch: 71, loss = 0.2965
epoch: 81, loss = 0.2788
epoch: 91, loss = 0.2639
accuracy= 0.9123
